<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Azure Managed Disks

## Azure Actions Covered

* Creating managed disks
* Appending managed disks to existing virtual machines

In this lecture, we're going to take a look at how to create managed disks and interact with them via the Python SDK.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). We'll add some new imports as well.

In [1]:
from azure.identity import AzureCliCredential
from azure.mgmt.compute import ComputeManagementClient
# New import
from azure.mgmt.compute.models import DiskCreateOption

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

We'll instantiate our credentials and the `ComputeManagementClient`. We don't need the `NetworkManagementClient` for this lecture.

In [2]:
credential = AzureCliCredential()
compute_client = ComputeManagementClient(credential, AZURE_SUBSCRIPTION_ID)

We can create a standalone disk via our `ComputeMangementClient`. You can specify parameters such as:
* `location` - Azure location for the disk
* `disk_size_gb` - Disk size in gigabytes
* `creation_data` - Creation data for the disk. For a full list of parameters, see [CreationData class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-compute/azure.mgmt.compute.v2022_07_02.models.creationdata?view=azure-python)

For full list of disk parameters (separate from `CreationData` parameters, see [Disk class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-compute/azure.mgmt.compute.v2022_07_02.models.disk?view=azure-python)

In [6]:
poller = compute_client.disks.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    disk_name='empty-managed-disk',
    parameters={
        'location': DEFAULT_LOCATION,
        'disk_size_gb': 20,
        'creation_data': {
            'create_option': DiskCreateOption.empty
        }
    }
)
disk_resource = poller.result()

Let's check the provisioning state of the disk to make sure it was created successfully.

In [7]:
disk_resource.provisioning_state

'Succeeded'

To attached the managed disk to a virtual machine, we have to retrieve both the desired VM and the desired disk via the `ComputeManagementClient`.

In [9]:
# Get virtual machine
vm = compute_client.virtual_machines.get(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    vm_name='test-vm'
)
# Get disk
managed_disk = compute_client.disks.get(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    disk_name='empty-managed-disk'
)

Now we can append the data disk to our existing VM. **Note:** This **only** updates the VM object in Python. We'll update our cloud service in the next step.

In [11]:
vm.storage_profile.data_disks.append({
    'lun': 12,
    'name': managed_disk.name,
    'create_option': DiskCreateOption.attach,
    'managed_disk': {
        'id': managed_disk.id
    }
})

Since we've appended the new managed disk to our Python VM object, we can pass that updated object to our `ComputeManagementClient` to update our VM in Azure.

In [12]:
async_update = compute_client.virtual_machines.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    vm_name=vm.name,
    # Updated VM parameters
    vm
)
async_update.wait()